In [ ]:
import codecademylib3_seaborn
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Make a request to the website to get the raw HTML
raw_data = requests.get('https://s3.amazonaws.com/codecademy-content/courses/beautifulsoup/cacao/index.html')
#print(raw_data.text)

In [ ]:
#Create a BeautifulSoup object to traverse this HTML
soup = BeautifulSoup(raw_data.content, 'html.parser')
print(soup)

In [ ]:
#How many terrible chocolate bars are out there? And how many earned a perfect 5? 
#The first thing to do is to put all of the ratings into a list.
#Loop through the ratings tags and get the text contained in each one. Add it to the ratings list.
rating_tags = soup.find_all(attrs={"class": "Rating"})
rating_tags = rating_tags[1:]
ratings = []
for tag in rating_tags:
  print(tag, float(tag.string)) #convert the rating to a float
  ratings.append(float(tag.string))
#print(ratings)

#Using Matplotlib, create a histogram of the ratings values
plt.hist(ratings)
#plt.show()

#Find the 10 most highly rated chocolatiers
#First, find all the tags on the webpage that contain the company names
name_tags = soup.select(".Company")
name_tags = name_tags[1:]
name = []
for tag in name_tags:
  print(tag, str(tag.string))
  name.append(str(tag.string))
print(name)

### try to see the unique values in the company list
def unique(name_list):
  unique_names = np.array(name_list)
  return np.unique(unique_names)

names = unique(name)
#print(names)

#Loop through the tags containing company names, and add the text from each tag to the list
d = {'Company': name, 'Rating': ratings}
Company_df = pd.DataFrame(data=d)
#print(Company_df)
 
#group your DataFrame by Company and take the average of the grouped ratings.Then, get the 10 highest rated chocolate companies. Print them out.
mean_rating = Company_df.groupby("Company").Rating.mean()
ten_best = mean_rating.nlargest(10)
#print(ten_best)

#want to see if the chocolate experts tend to rate chocolate bars with higher levels of cacao to be better than those with lower levels of cacao.
cocoa_percents = []
cocoa_percent_tags = soup.select(".CocoaPercent")

for tag in cocoa_percent_tags[1:]:
  percent = str(tag.get_text().strip('%'))
  cocoa_percents.append(percent)
print(cocoa_percents)

Company_df['CocoaPercentage'] = pd.DataFrame(data=cocoa_percents)
print(Company_df)

#Make a scatterplot of ratings vs percentage of cocoa
plt.figure(2)
plt.scatter(Company_df.CocoaPercentage, Company_df.Rating)

Company_df['CocoaPercentage'] = Company_df['CocoaPercentage'].apply(float)
#Company_df['Rating'] = Company_df['Rating'].apply(str)  


#use some numpy commands to draw a line of best-fit over the scatterplot.
z = np.polyfit(Company_df.CocoaPercentage, Company_df.Rating, 1)
line_function = np.poly1d(z)
plt.plot(Company_df.CocoaPercentage, line_function(Company_df.CocoaPercentage), "r--")
plt.show()